# Regression
This notebook contains codes for Ridge regression on a random dataset. Error function was optimised using scipy.optimise.minimsize
Sklearn fucntion for Lasso regression was used for comparing the results of lasso and ridge regression on same dataset(random). 

In [14]:
# Libraries
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from scipy.optimize import minimize

## Ridge Regression

In [15]:
def make_data(m,d):
#     set seed
    np.random.seed(35)
#     generate random matrix of size (150,75) from standard normal distribution
    X = np.random.randn(m,d)
    
#     generate true weight vector(random standard normal)
    t_theta = np.zeros((d,1))
    t_theta[0:10:2,0] = [10 for i in range(5)]
    t_theta[1:10:2,0] = [-10 for i in range(5)]
    
#     generate noise vector(mean = 0,standard deviation = 0.1)
    epsilon = np.random.randn(m,1)*0.1
    
#     make true label vector
    y = np.matmul(X,t_theta) + epsilon
        
    return(X,t_theta,y)

In [16]:
# generate dataset

data,t_theta,y = make_data(150,75)

# splitting data into train,test and validation sets
train_x = data[:80,:]
val_x = data[80:100,:]
test_x = data[100:150,:]

train_y = y[:80,:]
val_y = y[80:100,:]
test_y = y[100:150,:]

In [17]:
def get_theta(alpha):
    
#     calculate theta using closed form solution
    l = alpha
    theta = np.linalg.inv(np.matmul(train_x.T,train_x) + l * np.identity(len(train_x[0])))
    theta = np.matmul(theta,train_x.T)
    theta = np.matmul(theta,train_y)
    
    return(theta)

In [18]:
def get_loss(alpha):
    
#     calculate dummy y

#     calculate regularized loss value
    return(sum((val_y - np.matmul(val_x,get_theta(alpha)))**2))

In [19]:
# calculate optimal alpha value for ridge regression

# choose an initial guess for alpha
initial_guess = 10
opt_alpha = minimize(get_loss,initial_guess)
print("Minimum error will be achieved with aplha = ", opt_alpha.x[0])

# get final loss
final_theta = get_theta(opt_alpha.x[0])
loss = sum((test_y - np.matmul(test_x,final_theta))**2)
print("Final loss for alpha = opt_alpha will be: ", loss[0])

# built-in ridge regression
built_ridge = Ridge(alpha = opt_alpha.x,random_state = 25)
built_ridge.fit(train_x,train_y)
prediction = built_ridge.predict(test_x)
built_loss = sum((test_y-prediction)**2)
print("Final loss for alpha = opt_alpha for built in ridge regression will be: ", built_loss[0])

Minimum error will be achieved with aplha =  0.000934517964759
Final loss for alpha = opt_alpha will be:  10.8002447855
Final loss for alpha = opt_alpha for built in ridge regression will be:  10.6781499511


## Report for Ridge Regression

### Before setting threshold

In [20]:
true_zero = final_theta[10:]
true_nonzero = final_theta[:10]

true_zero[true_zero != 0] = 1
true_zero[true_zero == 0] = 0

true_nonzero[true_nonzero == 0] = 1
true_nonzero[true_nonzero != 0] = 0

print("Number of weights having true value zero but estimated to be non-zero: ", sum(true_zero)[0])
print("Number of weights having true value non-zero but estimated to be zero: ", sum(true_nonzero)[0])

Number of weights having true value zero but estimated to be non-zero:  65.0
Number of weights having true value non-zero but estimated to be zero:  0.0


### After setting threshold

In [21]:
threshold = 1
final_theta = get_theta(opt_alpha.x)
final_theta[abs(final_theta) < threshold] = 0
true_zero = final_theta[10:]
true_nonzero = final_theta[:10]

true_zero[true_zero != 0] = 1
true_zero[true_zero == 0] = 0

true_nonzero[true_nonzero == 0] = 1
true_nonzero[true_nonzero != 0] = 0

print("Number of weights having true value zero but estimated to be non-zero: ", sum(true_zero)[0])
print("Number of weights having true value non-zero but estimated to be zero: ", sum(true_nonzero)[0])

Number of weights having true value zero but estimated to be non-zero:  0.0
Number of weights having true value non-zero but estimated to be zero:  0.0


## Lasso 

In [23]:
# built-in lasso regression
built_lasso = Lasso(alpha = opt_alpha.x,random_state = 25)
built_lasso.fit(train_x,train_y)
prediction = built_lasso.predict(test_x)
built_loss = sum((test_y.T-prediction)**2)
print("Final loss for alpha = opt_alpha for built in lasso regression will be: ", built_loss[0])

Final loss for alpha = opt_alpha for built in lasso regression will be:  0.0175061631706


## Observation
1. Lasso regression is performing better than ridge. One of the reason for the same is, Lasso inherently performs feature selection as it can make coefficients zero since it uses direct sum of error terms. 
2. Accuracy increases after setting threshold.